In [1]:
from __future__ import division
from __future__ import print_function
from linear_gae.evaluation import get_roc_score, clustering_latent_space
from linear_gae.kcore import compute_kcore, expand_embedding
from linear_gae.model import *
from linear_gae.optimizer import OptimizerAE, OptimizerVAE
from linear_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time
import networkx as nx
import pickle as pkl
import sys
import pandas as pd

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')   # 添加的，不报错

In [3]:
# Select graph dataset
flags.DEFINE_string('dataset', 'power', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'link_prediction', 'Name of the learning task')

# Model
flags.DEFINE_string('model', 'gcn_ae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner
           product decoder

- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS
             workshop paper, with linear encoder, and inner product decoder

- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3
              of NeurIPS workshop paper, with Gaussian priors, linear encoders
              for mu and sigma, and inner product decoder
 
- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4
               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner
               product decoder
 
- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced
                in section 4 of NeurIPS workshop paper, with Gaussian priors,
                3-layer GCN encoders for mu and sigma, and inner product
                decoder
'''

' Available Models:\n\n- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer\n          GCN encoder and inner product decoder\n\n- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with\n           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner\n           product decoder\n\n- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS\n             workshop paper, with linear encoder, and inner product decoder\n\n- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3\n              of NeurIPS workshop paper, with Gaussian priors, linear encoders\n              for mu and sigma, and inner product decoder\n \n- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4\n               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner\n               product decoder\n \n- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced\n                

In [4]:
# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in encoder')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer(s).')
flags.DEFINE_integer('dimension', 16, 'Dimension of encoder output, i.e. \
                                       embedding dimension')

# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Link Prediction task)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Link Prediction task)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results at each epoch (for \
                                           Link Prediction task)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

# Parameters related to the "degeneracy framework" from IJCAI 2019 paper,
# aiming at scaling-up Graph AE/VAE by training the model only on the k-core
# (smaller) version of the graph, then expanding embedding to remaining nodes
# via simpler and faster heuristics
flags.DEFINE_boolean('kcore', False, 'Whether to run k-core decomposition \
                                      and use the framework. False = model \
                                      will be trained on the entire graph')
flags.DEFINE_integer('k', 2, 'Which k-core to use. Higher k => smaller graphs\
                              and faster (but maybe less accurate) training')
flags.DEFINE_integer('nb_iterations', 10, 'Number of fix point iterations in \
                                           algorithm 2 of IJCAI paper. See \
                                           kcore.py file for details')

# Lists to collect average results
if FLAGS.task == 'link_prediction':
    mean_roc = []
    mean_ap = []
    mean_acc = []
    mean_f1 = []

if FLAGS.kcore:
    mean_time_kcore = []
    mean_time_train = []
    mean_time_expand = []
    mean_core_size = []
mean_time = []

In [9]:
def load_data(filename_adj, nodes_numbers):
    raw_edges = pd.read_csv(filename_adj, header=None, sep='\t')
    
    drop_self_loop = raw_edges[raw_edges[0]!=raw_edges[1]]
    
    graph_np = np.zeros((nodes_numbers, nodes_numbers))
    
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0], drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1], drop_self_loop.iloc[i,0]]=1
        
    adj = nx.adjacency_matrix(nx.from_numpy_matrix(graph_np))
    
    features = sp.lil_matrix(np.eye(nodes_numbers))
    
    return adj, features

In [10]:
# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")

adj_name = 'datasets/graph.txt'
nodes_number = 2405
adj_init, features_init = load_data(adj_name, nodes_number)

Loading data...


In [11]:
# The entire training+test process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    if FLAGS.task == 'link_prediction' :
        if FLAGS.verbose:
            print("Masking test edges...")
        # Edge Masking for Link Prediction: compute Train/Validation/Test set
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)

    # Start computation of running times
    t_start = time.time()

    # Degeneracy Framework / K-Core Decomposition
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Starting k-core decomposition of the graph")
        # Save adjacency matrix of un-decomposed graph
        # (needed to embed nodes that are not in k-core, after GAE training)
        adj_orig = adj
        # Get the (smaller) adjacency matrix of the k-core subgraph,
        # and the corresponding nodes
        adj, nodes_kcore = compute_kcore(adj, FLAGS.k)
        # Get the (smaller) feature matrix of the nb_core graph
        if FLAGS.features:
            features = features_init[nodes_kcore,:]
        # Flag to compute k-core decomposition's running time
        t_core = time.time()
    elif FLAGS.features:
        features = features_init

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes,
                               features_nonzero)
    elif FLAGS.model == 'deep_gcn_ae':
        # Deep (3-layer GCN) Graph Autoencoder
        model = DeepGCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'deep_gcn_vae':
        # Deep (3-layer GCN) Graph Variational Autoencoder
        model = DeepGCNModelVAE(placeholders, num_features, num_nodes,
                                features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'linear_ae', 'deep_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'linear_vae', 'deep_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")

    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation, for Link Prediction
            if not FLAGS.kcore and FLAGS.validation and FLAGS.task == 'link_prediction':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = get_roc_score(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Flag to compute Graph AE/VAE training time
    t_model = time.time()

    # Compute embedding

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # If k-core is used, only part of the nodes from the original
    # graph are embedded. The remaining ones are projected in the
    # latent space via the expand_embedding heuristic
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Propagation to remaining nodes...")
        # Project remaining nodes in latent space
        emb = expand_embedding(adj_orig, emb, nodes_kcore, FLAGS.nb_iterations)
        # Compute mean running times for K-Core, GAE Train and Propagation steps
        mean_time_expand.append(time.time() - t_model)
        mean_time_train.append(t_model - t_core)
        mean_time_kcore.append(t_core - t_start)
        # Compute mean size of K-Core graph
        # Note: size is fixed if task is node clustering, but will vary if
        # task is link prediction due to edge masking
        mean_core_size.append(len(nodes_kcore))

    # Compute mean total running time
    mean_time.append(time.time() - t_start)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Link Prediction: classification edges/non-edges
    if FLAGS.task == 'link_prediction':
        # Get ROC and AP scores
        ap_score, roc_score, acc_score, f1_score = get_roc_score(test_edges, test_edges_false, emb)
        # Report scores
        mean_ap.append(ap_score)
        mean_roc.append(roc_score)
        mean_acc.append(acc_score)
        mean_f1.append(f1_score)

Masking test edges...
Preprocessing and Initializing...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
targets is deprecated, use labels instead

Training...
Epoch: 0001 train_loss= 0.73518 time= 0.29218
Epoch: 0002 train_loss= 0.73465 time= 0.15658
Epoch: 0003 train_loss= 0.73279 time= 0.16228
Epoch: 0004 train_loss= 0.72861 time= 0.15292
Epoch: 0005 train_loss= 0.72106 time= 0.15552
Epoch: 0006 train_loss= 0.70943 time= 0.15157
Epoch: 0007 train_loss= 0.69389 time= 0.15097
Epoch: 0008 train_loss= 0.67673 time= 0.15246
Epoch: 0009 train_loss= 0.66352 time= 0.15335
Epoch: 0010 train_loss= 0.66118 time= 0.15485
Epoch: 0011 train_loss= 0.66541 time= 0.15322
Epoch: 0012 train_loss= 0.66293 time= 0.15173
Epoch: 0013 train_loss= 0.65420 time= 0.15254
Epoch: 0014 train_loss= 0.64500 time= 0.14744
Epoch: 0015 train_loss= 0.63892 time= 0.15474
Epoch: 0016 train_loss= 0.63621 time= 0.16927
Epoch

Epoch: 0131 train_loss= 0.43246 time= 0.15901
Epoch: 0132 train_loss= 0.43213 time= 0.15293
Epoch: 0133 train_loss= 0.43182 time= 0.15510
Epoch: 0134 train_loss= 0.43154 time= 0.15343
Epoch: 0135 train_loss= 0.43127 time= 0.14926
Epoch: 0136 train_loss= 0.43102 time= 0.15907
Epoch: 0137 train_loss= 0.43077 time= 0.15234
Epoch: 0138 train_loss= 0.43052 time= 0.15589
Epoch: 0139 train_loss= 0.43026 time= 0.14767
Epoch: 0140 train_loss= 0.43000 time= 0.15513
Epoch: 0141 train_loss= 0.42974 time= 0.15221
Epoch: 0142 train_loss= 0.42946 time= 0.15858
Epoch: 0143 train_loss= 0.42919 time= 0.15374
Epoch: 0144 train_loss= 0.42891 time= 0.16114
Epoch: 0145 train_loss= 0.42863 time= 0.15371
Epoch: 0146 train_loss= 0.42835 time= 0.15401
Epoch: 0147 train_loss= 0.42808 time= 0.14911
Epoch: 0148 train_loss= 0.42782 time= 0.15142
Epoch: 0149 train_loss= 0.42756 time= 0.14997
Epoch: 0150 train_loss= 0.42731 time= 0.15680
Epoch: 0151 train_loss= 0.42707 time= 0.15080
Epoch: 0152 train_loss= 0.42683 ti

Epoch: 0109 train_loss= 0.44614 time= 0.18274
Epoch: 0110 train_loss= 0.44560 time= 0.17649
Epoch: 0111 train_loss= 0.44505 time= 0.17341
Epoch: 0112 train_loss= 0.44449 time= 0.18361
Epoch: 0113 train_loss= 0.44393 time= 0.16458
Epoch: 0114 train_loss= 0.44334 time= 0.18263
Epoch: 0115 train_loss= 0.44273 time= 0.17281
Epoch: 0116 train_loss= 0.44210 time= 0.16585
Epoch: 0117 train_loss= 0.44144 time= 0.16998
Epoch: 0118 train_loss= 0.44077 time= 0.17099
Epoch: 0119 train_loss= 0.44010 time= 0.16821
Epoch: 0120 train_loss= 0.43942 time= 0.17334
Epoch: 0121 train_loss= 0.43876 time= 0.16537
Epoch: 0122 train_loss= 0.43812 time= 0.16921
Epoch: 0123 train_loss= 0.43752 time= 0.16648
Epoch: 0124 train_loss= 0.43695 time= 0.15547
Epoch: 0125 train_loss= 0.43643 time= 0.17121
Epoch: 0126 train_loss= 0.43596 time= 0.16070
Epoch: 0127 train_loss= 0.43555 time= 0.17285
Epoch: 0128 train_loss= 0.43519 time= 0.18628
Epoch: 0129 train_loss= 0.43487 time= 0.17415
Epoch: 0130 train_loss= 0.43460 ti

Epoch: 0087 train_loss= 0.45207 time= 0.16042
Epoch: 0088 train_loss= 0.45132 time= 0.16591
Epoch: 0089 train_loss= 0.45061 time= 0.16442
Epoch: 0090 train_loss= 0.44992 time= 0.18067
Epoch: 0091 train_loss= 0.44924 time= 0.16836
Epoch: 0092 train_loss= 0.44858 time= 0.17669
Epoch: 0093 train_loss= 0.44792 time= 0.16953
Epoch: 0094 train_loss= 0.44727 time= 0.17254
Epoch: 0095 train_loss= 0.44662 time= 0.16365
Epoch: 0096 train_loss= 0.44598 time= 0.17489
Epoch: 0097 train_loss= 0.44534 time= 0.17561
Epoch: 0098 train_loss= 0.44469 time= 0.16906
Epoch: 0099 train_loss= 0.44402 time= 0.15866
Epoch: 0100 train_loss= 0.44334 time= 0.16830
Epoch: 0101 train_loss= 0.44264 time= 0.17300
Epoch: 0102 train_loss= 0.44194 time= 0.17002
Epoch: 0103 train_loss= 0.44124 time= 0.18202
Epoch: 0104 train_loss= 0.44055 time= 0.16927
Epoch: 0105 train_loss= 0.43988 time= 0.18198
Epoch: 0106 train_loss= 0.43924 time= 0.17235
Epoch: 0107 train_loss= 0.43863 time= 0.16454
Epoch: 0108 train_loss= 0.43806 ti

Epoch: 0065 train_loss= 0.48696 time= 0.16181
Epoch: 0066 train_loss= 0.48519 time= 0.17222
Epoch: 0067 train_loss= 0.48333 time= 0.17326
Epoch: 0068 train_loss= 0.48144 time= 0.16053
Epoch: 0069 train_loss= 0.47955 time= 0.16900
Epoch: 0070 train_loss= 0.47771 time= 0.15477
Epoch: 0071 train_loss= 0.47590 time= 0.17280
Epoch: 0072 train_loss= 0.47415 time= 0.17054
Epoch: 0073 train_loss= 0.47248 time= 0.17894
Epoch: 0074 train_loss= 0.47090 time= 0.16282
Epoch: 0075 train_loss= 0.46941 time= 0.17022
Epoch: 0076 train_loss= 0.46801 time= 0.17313
Epoch: 0077 train_loss= 0.46666 time= 0.17159
Epoch: 0078 train_loss= 0.46536 time= 0.16676
Epoch: 0079 train_loss= 0.46409 time= 0.17444
Epoch: 0080 train_loss= 0.46289 time= 0.17231
Epoch: 0081 train_loss= 0.46174 time= 0.17514
Epoch: 0082 train_loss= 0.46064 time= 0.16358
Epoch: 0083 train_loss= 0.45959 time= 0.17174
Epoch: 0084 train_loss= 0.45857 time= 0.16297
Epoch: 0085 train_loss= 0.45759 time= 0.17626
Epoch: 0086 train_loss= 0.45665 ti

...

Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 0.73518 time= 0.30293
Epoch: 0002 train_loss= 0.73465 time= 0.15780
Epoch: 0003 train_loss= 0.73279 time= 0.16354
Epoch: 0004 train_loss= 0.72860 time= 0.16491
Epoch: 0005 train_loss= 0.72103 time= 0.15679
Epoch: 0006 train_loss= 0.70931 time= 0.16310
Epoch: 0007 train_loss= 0.69363 time= 0.17003
Epoch: 0008 train_loss= 0.67632 time= 0.16229
Epoch: 0009 train_loss= 0.66305 time= 0.16867
Epoch: 0010 train_loss= 0.66084 time= 0.16125
Epoch: 0011 train_loss= 0.66532 time= 0.16606
Epoch: 0012 train_loss= 0.66288 time= 0.15659
Epoch: 0013 train_loss= 0.65405 time= 0.16157
Epoch: 0014 train_loss= 0.64480 time= 0.15857
Epoch: 0015 train_loss= 0.63877 time= 0.14960
Epoch: 0016 train_loss= 0.63620 time= 0.16134
Epoch: 0017 train_loss= 0.63534 time= 0.16637
Epoch: 0018 train_loss= 0.63441 time= 0.16375
Epoch: 0019 train_loss= 0.63249 time= 0.16865
Epoch: 0020 train_loss= 0.62945 time= 0.1

Epoch: 0179 train_loss= 0.42068 time= 0.17803
Epoch: 0180 train_loss= 0.42046 time= 0.16006
Epoch: 0181 train_loss= 0.42024 time= 0.17267
Epoch: 0182 train_loss= 0.42003 time= 0.16634
Epoch: 0183 train_loss= 0.41981 time= 0.17520
Epoch: 0184 train_loss= 0.41960 time= 0.17487
Epoch: 0185 train_loss= 0.41940 time= 0.16869
Epoch: 0186 train_loss= 0.41919 time= 0.17645
Epoch: 0187 train_loss= 0.41899 time= 0.17156
Epoch: 0188 train_loss= 0.41879 time= 0.15804
Epoch: 0189 train_loss= 0.41858 time= 0.17649
Epoch: 0190 train_loss= 0.41837 time= 0.16741
Epoch: 0191 train_loss= 0.41817 time= 0.16739
Epoch: 0192 train_loss= 0.41796 time= 0.16471
Epoch: 0193 train_loss= 0.41774 time= 0.15842
Epoch: 0194 train_loss= 0.41752 time= 0.16796
Epoch: 0195 train_loss= 0.41730 time= 0.16317
Epoch: 0196 train_loss= 0.41708 time= 0.16729
Epoch: 0197 train_loss= 0.41685 time= 0.16793
Epoch: 0198 train_loss= 0.41662 time= 0.16232
Epoch: 0199 train_loss= 0.41639 time= 0.17402
Epoch: 0200 train_loss= 0.41615 ti

Epoch: 0157 train_loss= 0.42268 time= 0.16275
Epoch: 0158 train_loss= 0.42240 time= 0.16781
Epoch: 0159 train_loss= 0.42213 time= 0.16277
Epoch: 0160 train_loss= 0.42186 time= 0.16622
Epoch: 0161 train_loss= 0.42158 time= 0.16988
Epoch: 0162 train_loss= 0.42130 time= 0.16412
Epoch: 0163 train_loss= 0.42102 time= 0.16311
Epoch: 0164 train_loss= 0.42074 time= 0.17026
Epoch: 0165 train_loss= 0.42047 time= 0.16892
Epoch: 0166 train_loss= 0.42021 time= 0.16400
Epoch: 0167 train_loss= 0.41995 time= 0.17261
Epoch: 0168 train_loss= 0.41970 time= 0.16559
Epoch: 0169 train_loss= 0.41947 time= 0.16270
Epoch: 0170 train_loss= 0.41924 time= 0.17226
Epoch: 0171 train_loss= 0.41903 time= 0.16067
Epoch: 0172 train_loss= 0.41882 time= 0.16833
Epoch: 0173 train_loss= 0.41862 time= 0.15310
Epoch: 0174 train_loss= 0.41843 time= 0.15071
Epoch: 0175 train_loss= 0.41825 time= 0.16735
Epoch: 0176 train_loss= 0.41807 time= 0.16143
Epoch: 0177 train_loss= 0.41790 time= 0.16057
Epoch: 0178 train_loss= 0.41773 ti

Epoch: 0135 train_loss= 0.43083 time= 0.15876
Epoch: 0136 train_loss= 0.43037 time= 0.15646
Epoch: 0137 train_loss= 0.42994 time= 0.16431
Epoch: 0138 train_loss= 0.42952 time= 0.16415
Epoch: 0139 train_loss= 0.42913 time= 0.16556
Epoch: 0140 train_loss= 0.42876 time= 0.17671
Epoch: 0141 train_loss= 0.42841 time= 0.17424
Epoch: 0142 train_loss= 0.42808 time= 0.17745
Epoch: 0143 train_loss= 0.42776 time= 0.17212
Epoch: 0144 train_loss= 0.42747 time= 0.17374
Epoch: 0145 train_loss= 0.42720 time= 0.17713
Epoch: 0146 train_loss= 0.42693 time= 0.17356
Epoch: 0147 train_loss= 0.42668 time= 0.16561
Epoch: 0148 train_loss= 0.42644 time= 0.16710
Epoch: 0149 train_loss= 0.42620 time= 0.16463
Epoch: 0150 train_loss= 0.42596 time= 0.17085
Epoch: 0151 train_loss= 0.42573 time= 0.17149
Epoch: 0152 train_loss= 0.42550 time= 0.16955
Epoch: 0153 train_loss= 0.42526 time= 0.17453
Epoch: 0154 train_loss= 0.42503 time= 0.16595
Epoch: 0155 train_loss= 0.42480 time= 0.18187
Epoch: 0156 train_loss= 0.42456 ti

Epoch: 0113 train_loss= 0.43946 time= 0.17453
Epoch: 0114 train_loss= 0.43891 time= 0.16636
Epoch: 0115 train_loss= 0.43837 time= 0.16770
Epoch: 0116 train_loss= 0.43784 time= 0.16716
Epoch: 0117 train_loss= 0.43733 time= 0.16336
Epoch: 0118 train_loss= 0.43684 time= 0.16473
Epoch: 0119 train_loss= 0.43636 time= 0.16308
Epoch: 0120 train_loss= 0.43590 time= 0.16656
Epoch: 0121 train_loss= 0.43545 time= 0.17224
Epoch: 0122 train_loss= 0.43502 time= 0.15889
Epoch: 0123 train_loss= 0.43460 time= 0.16878
Epoch: 0124 train_loss= 0.43419 time= 0.16357
Epoch: 0125 train_loss= 0.43379 time= 0.16959
Epoch: 0126 train_loss= 0.43340 time= 0.16761
Epoch: 0127 train_loss= 0.43301 time= 0.15402
Epoch: 0128 train_loss= 0.43263 time= 0.17288
Epoch: 0129 train_loss= 0.43225 time= 0.15956
Epoch: 0130 train_loss= 0.43188 time= 0.17249
Epoch: 0131 train_loss= 0.43150 time= 0.16834
Epoch: 0132 train_loss= 0.43112 time= 0.17425
Epoch: 0133 train_loss= 0.43073 time= 0.16609
Epoch: 0134 train_loss= 0.43034 ti

In [12]:
###### Report Final Results ######

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

if FLAGS.task == 'link_prediction':
    print("AP scores\n", mean_ap)
    print("Mean AP score: ", np.mean(mean_ap),
          "\nStd of AP scores: ", np.std(mean_ap), "\n \n")
    
    print("AUC scores\n", mean_roc)
    print("Mean AUC score: ", np.mean(mean_roc),
          "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")
    
    print("ACC scores\n", mean_acc)
    print("Mean ACC score: ", np.mean(mean_acc),
          "\nStd of ACC scores: ", np.std(mean_acc), "\n \n")
    
    print("F1 scores\n", mean_f1)
    print("Mean F1 score: ", np.mean(mean_f1),
          "\nStd of F1 scores: ", np.std(mean_f1), "\n \n")

if FLAGS.kcore:
    print("Details on degeneracy framework, with k =", FLAGS.k, ": \n \n")

    print("Running times for k-core decomposition\n", mean_time_kcore)
    print("Mean: ", np.mean(mean_time_kcore),
          "\nStd: ", np.std(mean_time_kcore), "\n \n")

    print("Running times for autoencoder training\n", mean_time_train)
    print("Mean: ", np.mean(mean_time_train),
          "\nStd: ", np.std(mean_time_train), "\n \n")

    print("Running times for propagation\n", mean_time_expand)
    print("Mean: ", np.mean(mean_time_expand),
          "\nStd: ", np.std(mean_time_expand), "\n \n")

    print("Sizes of k-core subgraph\n", mean_core_size)
    print("Mean: ", np.mean(mean_core_size),
          "\nStd: ", np.std(mean_core_size), "\n \n")


Test results for gcn_ae model on power on link_prediction 
 ___________________________________________________

AP scores
 [0.9333834164583548, 0.9248066887832662, 0.9398935085906264, 0.9319432577246792, 0.9296344382520424, 0.9331233971140167, 0.9284450054033184, 0.9328227795145767, 0.9356651873439025, 0.9410411466179655]
Mean AP score:  0.933075882580275 
Std of AP scores:  0.004687662725212013 
 

AUC scores
 [0.9130394161757667, 0.8990959449288718, 0.9170270405075336, 0.9058934802174674, 0.9068043097460621, 0.9109635288521165, 0.9083360071347691, 0.910599494819029, 0.9101245383505014, 0.9183015318462778]
Mean AUC score:  0.9100185292578397 
Std of AUC scores:  0.005268546939201213 
 

ACC scores
 [0.7424503882657464, 0.7333908541846419, 0.7489214840379638, 0.7402933563416738, 0.734253666954271, 0.722174288179465, 0.7303710094909405, 0.7450388265746333, 0.7364106988783434, 0.7277825711820535]
Mean ACC score:  0.7361087144089731 
Std of ACC scores:  0.007803687187814445 
 

F1 score